# *Script* de Treino

## Instalação de dependências

In [1]:
import torch
torch.cuda.empty_cache()

In [2]:
%%capture
!pip install OmegaConf
!pip install diffusers
!pip install ftfy
!pip install accelerate==0.16.0
!pip install tensorflow-gpu
!pip install pytorch_lightning
!pip install bitsandbytes
!pip install nvidia-pyindex
!pip install --upgrade nvidia-tensorrt
!pip install --upgrade transformers

## Clone do repositório 
#### Repositório que contém as *scripts* do Dreambooth

In [3]:
!git clone https://github.com/CrazyBoyM/dreambooth-for-diffusion.git

Cloning into 'dreambooth-for-diffusion'...
remote: Enumerating objects: 105, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 105 (delta 22), reused 20 (delta 20), pack-reused 78
Receiving objects: 100% (105/105), 84.47 KiB | 910.00 KiB/s, done.
Resolving deltas: 100% (49/49), done.


#### Mudar *path* para a pasta clonada

In [4]:
%cd /kaggle/working/dreambooth-for-diffusion

/kaggle/working/dreambooth-for-diffusion


## Modelo
#### Download do checkpoint do modelo Stable diffusion 1.4 original da CompVis

In [5]:
!wget https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4-full-ema.ckpt -P ./ckpt_models

--2023-06-10 18:15:13--  https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4-full-ema.ckpt
Resolving huggingface.co (huggingface.co)... 13.35.35.43, 13.35.35.73, 13.35.35.36, ...
Connecting to huggingface.co (huggingface.co)|13.35.35.43|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/4c/37/4c372b4ebb57bbd02e68413d4951aa326d4b3cfb6e62db989e529c6d4b26fb21/14749efc0ae8ef0329391ad4436feb781b402f4fece4883c7ad8d10556d8a36a?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27sd-v1-4-full-ema.ckpt%3B+filename%3D%22sd-v1-4-full-ema.ckpt%22%3B&Expires=1686680060&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzRjLzM3LzRjMzcyYjRlYmI1N2JiZDAyZTY4NDEzZDQ5NTFhYTMyNmQ0YjNjZmI2ZTYyZGI5ODllNTI5YzZkNGIyNmZiMjEvMTQ3NDllZmMwYWU4ZWYwMzI5MzkxYWQ0NDM2ZmViNzgxYjQwMmY0ZmVjZTQ4ODNjN2FkOGQxMDU1NmQ4YTM2YT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoiLCJDb25kaXRp

#### Tornar o checkpoint do modelo compativel com a livraria de difusores do Huggingface

In [10]:
!python tools/ckpt2diffusers.py \
    --checkpoint_path=./ckpt_models/sd-v1-4-full-ema.ckpt \
    --dump_path=./model \
    --original_config_file=./ckpt_models/model.yaml \
    --scheduler_type="ddim"

Checkpoint ./ckpt_models/sd-v1-4-full-ema.ckpt has both EMA and non-EMA weights.
In this conversion only the non-EMA weights are extracted. If you want to instead extract the EMA weights (usually better for inference), please make sure to add the `--extract_ema` flag.
╭───────────────────── Traceback (most recent call last) ──────────────────────╮
│ /kaggle/working/dreambooth-for-diffusion/tools/ckpt2diffusers.py:808 in      │
│ <module>                                                                     │
│                                                                              │
│   805 │   │   converted_vae_checkpoint = convert_ldm_vae_checkpoint(checkpoi │
│   806 │                                                                      │
│   807 │   vae = AutoencoderKL(**vae_config)                                  │
│ ❱ 808 │   vae.load_state_dict(converted_vae_checkpoint)                      │
│   809 │                                                                      │
│ 

### Criar pasta para guardar dataset

In [7]:
!mkdir datasets/data

## Dataset
#### Fazer download do dataset modern-logo-dataset de logo-wizard e guardar cada imagem sendo o nome do ficheiro a sua descrição

In [8]:
from datasets import load_dataset
from PIL import Image
import os
import io

dataset = load_dataset("logo-wizard/modern-logo-dataset")
output_folder = 'datasets/data'

for i, example in enumerate(dataset["train"]):
    image = example["image"]
    image.save(os.path.join(output_folder, example["text"][:min(len(example["text"]), 200)].replace("/","-")+".png"))

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/logo-wizard--modern-logo-dataset-88f786d382f37e89/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

## Fine-tuning
#### Fazer o fine-tuning do modelo em meia precisão com resolução 512x512 no dataset escolhido com a class "logo", learning rate de 1e-6, batch size de 1 e 1000 train steps

In [9]:
%%time
!accelerate launch tools/train_dreambooth.py \
  --pretrained_model_name_or_path="./model"  \
  --mixed_precision="fp16" \
  --instance_data_dir="/kaggle/working/dreambooth-for-diffusion/datasets/data" \
  --class_data_dir="/kaggle/working/dreambooth-for-diffusion/datasets/data" \
  --class_prompt="logo" \
  --use_8bit_adam \
  --use_filename_as_label \
  --output_dir="./new_model" \
  --center_crop \
  --resolution=512 \
  --train_batch_size=1 \
  --use_8bit_adam \
  --gradient_accumulation_steps=1 --gradient_checkpointing \
  --learning_rate=1e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=1000 \
  --save_model_every_n_steps=1000

/opt/conda/lib/python3.7/site-packages/accelerate/accelerator.py:233: FutureWarning: `logging_dir` is deprecated and will be removed in version 0.18.0 of 🤗 Accelerate. Use `project_dir` instead.
  FutureWarning,
╭───────────────────── Traceback (most recent call last) ──────────────────────╮
│ /kaggle/working/dreambooth-for-diffusion/tools/train_dreambooth.py:784 in    │
│ <module>                                                                     │
│                                                                              │
│   781                                                                        │
│   782 if __name__ == "__main__":                                             │
│   783 │   args = parse_args()                                                │
│ ❱ 784 │   main(args)                                                         │
│   785                                                                        │
│                                                          

## Teste 
#### Testar o modelo ao gerar 3 imagens para cada prompt como forma de testar o modelo

In [ ]:
from diffusers import StableDiffusionPipeline
import torch
import matplotlib.pyplot as plt
from diffusers import DDIMScheduler

model_path = "./new_model"
prompt = "a logo of a babershop, a man beard, black foreground, white background, minimalism, modern"
prompt = "a logo of a coffeeshop, a red mug, red foreground, white background, minimalism, modern"
prompt = "a logo of a parrot, minimalism, modern"
prompt = "a logo of a cow, minimalism, modern"
prompt = "a logo of a car company, front view of a blue car, minimalism, modern"

pipe = StableDiffusionPipeline.from_pretrained(
        model_path, 
        torch_dtype=torch.float16,
        scheduler=DDIMScheduler(
            beta_start=0.00085,
            beta_end=0.012,
            beta_schedule="scaled_linear",
            clip_sample=False,
            set_alpha_to_one=True,
        ),
        safety_checker=None
    )

pipe = pipe.to("cuda")
images = pipe(prompt, num_inference_steps=50, guidance_scale=15.0, num_images_per_prompt=3).images
for i, image in enumerate(images):
    plt.subplot(1,3,i+1)
    plt.imshow(image)
    image.save(f"test-{i}.png")

#### Zipar o modelo para fazer download

In [ ]:
!zip -r logo_model new_model